# SI 618 - Hypothesis testing (t-test and ANOVA)

Version 2021.02.16.1.CT

Today's class will focus on two common statistical techniques used to investigate
hypotheses about the mean values within samples.  We'll spend most of our time 
focusing on ANOVA, as that seems to be scarier than the more familiar t-test.

We will be using two datasets: a simple one about pizza dough recipes and, during our live session, 
a more complex one drawn from a FiveThirtyEight example about biographical films.

We'll start by importing the usual suspects (including scipy, which we haven't used
much so far):

In [ ]:
import numpy as np
import pandas as pd
import scipy

And we'll also import the main packages that we'll be using for our statistical analyses:

In [ ]:
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Pizza Dough Recipes

https://dasl.datadescription.com/datafile/activating-baking-yeast/?_sfm_methods=Analysis+of+Variance&_sfm_cases=4+59943

To shorten the time it takes him to
make his favorite pizza, a student designed an experiment to
test the effect of sugar and milk on the activation times for
baking yeast. Specifically, he tested four different recipes and
measured how many seconds it took for the same amount of
dough to rise to the top of a bowl. He randomized the order
of the recipes and replicated each treatment 4 times.

Let's go ahead and read in the data/activating-baking-yeast.txt file into a DataFrame called ```yeast```

In [ ]:
yeast = pd.read_csv('https://raw.githubusercontent.com/umsi-data-science/si370/master/data/activating-baking-yeast.txt',sep='\t')

As usual, inspect the DataFrame so we know what we're dealing with:

In [ ]:
yeast.head()

In [ ]:
yeast.columns

Those column names aren't nice to work with, so let's go ahead and rename them
to ```activation``` and ```recipe```

In [ ]:
to_be_renamed = {"Activation Times" :"activation", "Recipe ": "recipe"}
yeast = yeast.rename(columns = to_be_renamed)  

In [ ]:
yeast.head()

Let's start by visually examining our data:

In [ ]:
import seaborn as sns

### Q2: Create a boxplot the looks like the following:
![](resources/pizza-boxplot.png)

In [ ]:
sns.boxplot(x="recipe",y="activation", data=yeast)

## Let's do an ANOVA!
We want to know if the above differences are statistically "real".  In other words, 
we want to know if the activation times vary according to the recipes.

We use statsmodels.forumula.api to create the model in a "readable" way.  For example, 
```activation ~ recipe``` would do that for us.  So we're going to create a model,
fit it to the data, and examine it.  We'll talk about the "Type 2" bit in class.

In [ ]:
yeast_lm = ols('activation ~ recipe', data=yeast).fit()
table = sm.stats.anova_lm(yeast_lm, typ=2) # Type 2 ANOVA DataFrame
table

### What does that tell us?

Where do all those numbers come from?  Let's walk through an ANOVA by hand:
(this material based on https://jooskorstanje.com/1_Way_ANOVA_Pizza_Delivery.html)

In [ ]:
yeast.head()

In [ ]:
yeast.groupby('recipe').mean()

In [ ]:
# compute overall mean

overall_mean = yeast['activation'].mean()
overall_mean

In [ ]:
# compute sum of squares total
yeast['overall_mean'] = overall_mean
ss_total = sum((yeast['activation'] - yeast['overall_mean'])**2)
ss_total

In [ ]:
# compute group means
group_means = yeast.groupby('recipe').mean()
group_means = group_means.rename(columns = {'activation': 'group_mean'})
group_means

In [ ]:
# add group means and overall mean to the original data frame
yeast = yeast.merge(group_means['group_mean'], left_on = 'recipe', right_index = True)
yeast

In [ ]:
# compute sum of squares residual
ss_residual = sum((yeast['activation'] - yeast['group_mean'])**2)
ss_residual

In [ ]:
# compute Sum of Squares Model
ss_explained = sum((yeast['overall_mean'] - yeast['group_mean'])**2)
ss_explained

In [ ]:
# compute Mean Square Residual
n_groups = len(set(yeast['recipe']))
n_obs = yeast.shape[0]
df_residual = n_obs - n_groups
ms_residual = ss_residual / df_residual
ms_residual

In [ ]:
# compute Mean Square Explained
df_explained = n_groups - 1
ms_explained = ss_explained / df_explained
ms_explained

In [ ]:
# compute F-Value
f = ms_explained / ms_residual
f

In [ ]:
# compute p-value
import scipy.stats
p_value = 1 - scipy.stats.f.cdf(f, df_explained, df_residual)
p_value

In [ ]:
# (from our ANOVA way above)
table

## Tukey's Honestly Significant Differences (HSD)

Ok, so we know there are significant differences between the different recipes, but
which one(s) are different from other ones?  To answer that, we can use Tukey's HSD (Honestly Sigificant Differences):

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
res2 = pairwise_tukeyhsd(yeast['activation'], yeast['recipe'])
res2.summary()